# Microsoft Fabric API - Authentication with Service Principal

This notebook demonstrates how to use the `microsoft-fabric-api` package to authenticate and interact with Microsoft Fabric using a Service Principal.

## Step 1: Install the Package

In [2]:
%pip install microsoft-fabric-api azure-identity

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached charset_normalizer-3.4.4-cp314-cp314-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.6.3-py3-none-any.whl.metadata (6.9 kB)
  Using cached certifi-2026.1.4-py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ------------------- -------------------- 1.0/2.1 MB 6.1 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 6.1 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 6.1 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 6.1 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 6.1 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 6.1 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 6.1 M

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Step 2: Import Required Libraries

In [3]:
import os

from azure.identity import ClientSecretCredential
from microsoft_fabric_api import FabricClient

## Step 3: Configure Service Principal Credentials

Set your Azure Service Principal credentials. These should match the GitHub secrets used in your CI/CD pipeline:
- `AZURE_CLIENT_ID`: Application (client) ID
- `AZURE_CLIENT_SECRET`: Client secret value
- `AZURE_TENANT_ID`: Directory (tenant) ID

**Security Note**: Never commit credentials to source control. Use environment variables or secure secret management.

In [ ]:
# Option 1: Load from environment variables (recommended)
client_id = os.getenv("AZURE_CLIENT_ID", "your-client-id-here")
client_secret = os.getenv("AZURE_CLIENT_SECRET", "your-client-secret-here")
tenant_id = os.getenv("AZURE_TENANT_ID", "your-tenant-id-here")

# Option 2: Directly set values (for testing only - DO NOT commit)
# client_id = "your-client-id-here"
# client_secret = "your-client-secret-here"
# tenant_id = "your-tenant-id-here"

print(f"Client ID configured: {client_id[:8]}..." if len(client_id) > 8 else "Not configured")
print(f"Tenant ID configured: {tenant_id[:8]}..." if len(tenant_id) > 8 else "Not configured")

Client ID configured: 6e9a5f44...
Tenant ID configured: ac68a9ec...


## Step 4: Authenticate with Service Principal

In [13]:
# Create Azure credential using Service Principal
credential = ClientSecretCredential(tenant_id=tenant_id, client_id=client_id, client_secret=client_secret)

# Initialize Fabric API client
fabric_client = FabricClient(token_credential=credential)

print("✓ Successfully authenticated with Microsoft Fabric API")

✓ Successfully authenticated with Microsoft Fabric API


## Step 5: Test Authentication - List Workspaces

Test the connection by listing all workspaces the Service Principal has access to.

In [14]:
# List all workspaces
workspaces = list(fabric_client.core.workspaces.list_workspaces())

print(f"Found {len(workspaces)} workspace(s):\n")
for ws in workspaces:
    print(f"  • {ws.display_name} (ID: {ws.id})")

Found 7 workspace(s):

  • [D] Fabric Blueprint (ID: 8b09b6cd-e0ca-497f-b2e1-f9b71ccd4420)
  • [T] Fabric Blueprint (ID: 3c9daf2c-6d1f-430d-93a8-f1d8cb1a6da0)
  • [P] Fabric Blueprint (ID: 1e9888c0-2189-4acd-b03e-bb6074c40255)
  • [D] Fabric Bronze (ID: f9bcaf15-7d93-4040-96b7-e444730316dc)
  • [D] Fabric Autocreated Workspace (ID: 574767fb-89ca-4b66-86fa-4afb8aa03ddf)
  • [T] Fabric Autocreated Workspace (ID: 4b203ed4-5b7a-4e78-b01d-26a87a3bb5e8)
  • [T] Fabric Bronze (ID: 6598cfe4-a2c1-436f-9a77-dd9b1dda3216)


## Example: Get Workspace Items

Retrieve all items from a specific workspace.

In [19]:
# Replace with your workspace name or ID
workspace_name = "[D] Fabric Blueprint"  # Example from config.yml

# Find workspace by name
target_workspace = None
for ws in workspaces:
    if ws.display_name == workspace_name:
        target_workspace = ws
        break

if target_workspace:
    # Get items in workspace
    items = list(fabric_client.core.items.list_items(workspace_id=target_workspace.id))

    print(f"\nItems in workspace '{workspace_name}':\n")
    for item in items:
        print(f"  • {item.display_name} ({item.type})")
else:
    print(f"Workspace '{workspace_name}' not found.")


Items in workspace '[D] Fabric Blueprint':

  • lakehouse_bronze (SQLEndpoint)
  • lakehouse_gold (SQLEndpoint)
  • lakehouse_silver (SQLEndpoint)
  • lakehouse_bronze (Lakehouse)
  • lakehouse_gold (Lakehouse)
  • lakehouse_silver (Lakehouse)
  • env (Environment)
  • nb_gd_modeling (Notebook)
  • nb_sl_transform (Notebook)
  • cp_br_source (CopyJob)
  • da_agent (DataAgent)


## API Discovery

Since the documentation is limited, let's explore the API structure programmatically.

In [22]:
# Explore the FabricClient structure
print("FabricClient attributes:")
print("=" * 60)
for attr in dir(fabric_client):
    if not attr.startswith("_"):
        obj = getattr(fabric_client, attr)
        print(f"  • {attr}: {type(obj).__name__}")

FabricClient attributes:
  • admin: FabricAdminClient
  • anomalydetector: FabricAnomalyDetectorClient
  • apacheairflowjob: FabricApacheAirflowJobClient
  • copyjob: FabricCopyJobClient
  • core: FabricCoreClient
  • cosmosdbdatabase: FabricCosmosDBDatabaseClient
  • dashboard: FabricDashboardClient
  • dataflow: FabricDataflowClient
  • datamart: FabricDatamartClient
  • datapipeline: FabricDataPipelineClient
  • digitaltwinbuilder: FabricDigitalTwinBuilderClient
  • digitaltwinbuilderflow: FabricDigitalTwinBuilderFlowClient
  • environment: FabricEnvironmentClient
  • eventhouse: FabricEventhouseClient
  • eventstream: FabricEventstreamClient
  • graphmodel: FabricGraphModelClient
  • graphqlapi: FabricGraphQLApiClient
  • graphqueryset: FabricGraphQuerySetClient
  • kqldashboard: FabricKQLDashboardClient
  • kqldatabase: FabricKQLDatabaseClient
  • kqlqueryset: FabricKQLQuerysetClient
  • lakehouse: FabricLakehouseClient
  • map: FabricMapClient
  • mirroredazuredatabrickscatalog: 

In [23]:
# Explore the 'core' module methods
if hasattr(fabric_client, "core"):
    print("\nfabric_client.core attributes:")
    print("=" * 60)
    for attr in dir(fabric_client.core):
        if not attr.startswith("_"):
            obj = getattr(fabric_client.core, attr)
            print(f"  • core.{attr}: {type(obj).__name__}")


fabric_client.core attributes:
  • core.capacities: CapacitiesOperations
  • core.close: method
  • core.connections: ConnectionsOperations
  • core.deployment_pipelines: DeploymentPipelinesOperations
  • core.deploymentpipelines: DeploymentPipelinesOperations
  • core.domains: DomainsOperations
  • core.external_data_shares_provider: ExternalDataSharesProviderOperations
  • core.external_data_shares_recipient: ExternalDataSharesRecipientOperations
  • core.folders: FoldersOperations
  • core.gateways: GatewaysOperations
  • core.git: GitOperations
  • core.items: ItemsOperations
  • core.job_scheduler: JobSchedulerOperations
  • core.long_running_operations: LongRunningOperationsOperations
  • core.managed_private_endpoints: ManagedPrivateEndpointsOperations
  • core.one_lake_data_access_security: OneLakeDataAccessSecurityOperations
  • core.one_lake_settings: OneLakeSettingsOperations
  • core.one_lake_shortcuts: OneLakeShortcutsOperations
  • core.onelakesettings: OneLakeSettingsOp

In [24]:
# Explore workspaces operations
if hasattr(fabric_client.core, "workspaces"):
    print("\nfabric_client.core.workspaces methods:")
    print("=" * 60)
    for attr in dir(fabric_client.core.workspaces):
        if not attr.startswith("_") and callable(getattr(fabric_client.core.workspaces, attr)):
            print(f"  • workspaces.{attr}()")


fabric_client.core.workspaces methods:
  • workspaces.add_workspace_role_assignment()
  • workspaces.assign_to_capacity()
  • workspaces.assign_to_domain()
  • workspaces.begin_deprovision_identity()
  • workspaces.begin_provision_identity()
  • workspaces.create_workspace()
  • workspaces.delete_workspace()
  • workspaces.delete_workspace_role_assignment()
  • workspaces.deprovision_identity()
  • workspaces.get_git_outbound_policy()
  • workspaces.get_network_communication_policy()
  • workspaces.get_outbound_cloud_connection_rules()
  • workspaces.get_outbound_gateway_rules()
  • workspaces.get_workspace()
  • workspaces.get_workspace_role_assignment()
  • workspaces.list_workspace_role_assignments()
  • workspaces.list_workspaces()
  • workspaces.provision_identity()
  • workspaces.set_git_outbound_policy()
  • workspaces.set_network_communication_policy()
  • workspaces.set_outbound_cloud_connection_rules()
  • workspaces.set_outbound_gateway_rules()
  • workspaces.unassign_from_

In [25]:
# Explore items operations
if hasattr(fabric_client.core, "items"):
    print("\nfabric_client.core.items methods:")
    print("=" * 60)
    for attr in dir(fabric_client.core.items):
        if not attr.startswith("_") and callable(getattr(fabric_client.core.items, attr)):
            print(f"  • items.{attr}()")


fabric_client.core.items methods:
  • items.begin_create_item()
  • items.begin_get_item_definition()
  • items.begin_update_item_definition()
  • items.bulk_move_items()
  • items.create_item()
  • items.delete_item()
  • items.get_item()
  • items.get_item_definition()
  • items.list_item_connections()
  • items.list_items()
  • items.move_item()
  • items.update_item()
  • items.update_item_definition()


In [26]:
# Get help documentation for a specific method
import inspect

# Example: Get signature and docstring for list_workspaces
method = fabric_client.core.workspaces.list_workspaces
print("\nMethod: list_workspaces")
print("=" * 60)
print(f"Signature: {inspect.signature(method)}")
print(f"\nDocstring:\n{inspect.getdoc(method)}")


Method: list_workspaces
Signature: (*, roles: str | None = None, continuation_token_parameter: str | None = None, prefer_workspace_specific_endpoints: bool | None = None, **kwargs: Any) -> Iterable[ForwardRef('_models.Workspace')]

Docstring:
Returns a list of workspaces the principal can access. Use the roles query parameter to filter
results by the principal workspace role.

This API supports `pagination </rest/api/fabric/articles/pagination>`_.

Required Delegated Scopes
-------------------------

Workspace.Read.All or Workspace.ReadWrite.All

Microsoft Entra supported identities
------------------------------------

This API supports the Microsoft `identities </rest/api/fabric/articles/identity-support>`_
listed in this section.

.. list-table::
   :header-rows: 1

   * - Identity
     - Support
   * - User
     - Yes
   * - `Service principal
</entra/identity-platform/app-objects-and-service-principals#service-principal-object>`_ and
`Managed identities </entra/identity/managed-i

In [27]:
# Get help for list_items method
method = fabric_client.core.items.list_items
print("\nMethod: list_items")
print("=" * 60)
print(f"Signature: {inspect.signature(method)}")
print(f"\nDocstring:\n{inspect.getdoc(method)}")


Method: list_items
Signature: (workspace_id: str, *, type: str | None = None, recursive: bool | None = None, root_folder_id: str | None = None, continuation_token_parameter: str | None = None, **kwargs: Any) -> Iterable[ForwardRef('_models.Item')]

Docstring:
Returns a list of items from the specified workspace.

This API supports `pagination </rest/api/fabric/articles/pagination>`_.

Permissions
-----------

 The caller must have a *viewer* workspace role.

Required Delegated Scopes
-------------------------

Workspace.Read.All or Workspace.ReadWrite.All

Microsoft Entra supported identities
------------------------------------

This API supports the Microsoft `identities </rest/api/fabric/articles/identity-support>`_
listed in this section.

.. list-table::
   :header-rows: 1

   * - Identity
     - Support
   * - User
     - Yes
   * - `Service principal
</entra/identity-platform/app-objects-and-service-principals#service-principal-object>`_ and
`Managed identities </entra/identity

## Documentation Resources

Based on the PyPI page, here are the key resources:

1. **Microsoft Fabric REST API documentation**: https://learn.microsoft.com/en-us/rest/api/fabric
2. **Package Version**: 0.1.0b2 (beta release - still evolving)
3. **Authentication Guide**: https://learn.microsoft.com/en-us/rest/api/fabric/articles/get-started/fabric-api-quickstart

**Key Pattern**: The SDK follows the REST API structure:
- `fabric_client.core.*` - Maps to Core API endpoints
- Methods generally return `ItemPaged` objects (convert to `list()` for iteration)
- Uses `TokenCredential` from `azure-identity` for authentication

Since this is a beta package wrapping the REST API, you can reference the REST API docs to understand available operations.